In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy

In [ ]:
def prob_MGDD(x):

    return 1 / (1 + np.exp(-0.0120*(x-975)))

def prob_CDD(x):
    
    return 2e7/(2e7 + x**3)

#This is the simulation algorithm for 1 time step only (just for testing purposes)
def disease_simulation(filename_MGDD, filename_CDD, R0=5.0, γ=1.0/5.0, I0=1.0, use_vector=False, 
                       filename_vector="Data/vector_Europe.txt"):
    
    I = I0
    
    if use_vector == True:
        
        vector, lons_v, lats_v = np.loadtxt(filename_vector, unpack=True)

        R0_f = R0 * vector/1000
        
        R0_f[R0_f <= 1] = 1.00001 #
        
    else:
        
        R0_f = R0
        
    MGDD, lons, lats = np.loadtxt(filename_MGDD, unpack=True)
    CDD, lons, lats = np.loadtxt(filename_CDD, unpack=True)

    prob_1 = prob_MGDD(MGDD)
    prob_2 = prob_CDD(CDD)

    prob_f = prob_1 * prob_2

    I = I * prob_f * np.exp((R0_f-1)*γ)
    
    risk = np.log(I) / ((R0_f-1)*γ*1)
    
    risk[risk < -1.0] = -1.0
    
    return risk, lons, lats

# Simulate PD progression in 2019 

In [ ]:
filename_MGDD = "Data/Europe_2019_hot.txt"
filename_CDD = "Data/Europe_2019_cold.txt"

use_vector = True

risk, lons, lats = disease_simulation(filename_MGDD, filename_CDD, use_vector=use_vector)

# Plot 

In [ ]:
Europe_shape = (471, 891) 

risk = np.reshape(risk, Europe_shape)
lons = np.reshape(lons, Europe_shape)
lats = np.reshape(lats, Europe_shape)

In [ ]:
clevels = np.linspace(-1, 1, 100)
cbar_ticks = np.arange(-1, 1.2, 0.2)

cmap = 'jet'

#-- create figure and axes instances
fig = plt.figure(figsize=(11, 11), dpi=100)
ax  = fig.add_axes([0.1,0.1,0.8,0.9])

projection = ccrs.PlateCarree()

#-- create map
ax = plt.axes(projection=projection) 

#-- add map features
ax.coastlines(resolution='10m') #110m, 50m, 10m
ax.add_feature(cartopy.feature.LAND, edgecolor='black')
ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black')
ax.add_feature(cartopy.feature.OCEAN, zorder=100, edgecolor='black', facecolor='azure')

cnplot = ax.contourf(lons, lats, risk, clevels, cmap=cmap)

cbar = plt.colorbar(cnplot, ticks=cbar_ticks, shrink=0.42, pad = 0.03)

gl = ax.gridlines(crs=projection, linewidth=1, color='gray', alpha=0.5, zorder=200, linestyle='--')

gl.xlabel_style = {'size': 10, 'color': 'gray'}
gl.ylabel_style = {'size': 10, 'color': 'gray'}

gl.bottom_labels = True
gl.left_labels = True

cbar.set_label("Risk", fontsize=30)

plt.savefig("Europe_risk_test_vector.png", dpi=300, bbox_inches='tight')

## Zoom a given region 

In [ ]:
clevels = np.linspace(-1, 1, 100)
cbar_ticks = np.arange(-1, 1.2, 0.2)

cmap = 'jet'

zoom_coords = [-10, 5, 35, 45] #SPAIN

#-- create figure and axes instances
fig = plt.figure(figsize=(11, 11), dpi=100)
ax  = fig.add_axes([0.1,0.1,0.8,0.9])

projection = ccrs.PlateCarree()

#-- create map
ax = plt.axes(projection=projection) 

#-- add map features
ax.coastlines(resolution='10m') #110m, 50m, 10m
ax.add_feature(cartopy.feature.LAND, edgecolor='black')
ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black')
ax.add_feature(cartopy.feature.OCEAN, zorder=100, edgecolor='black', facecolor='azure')

cnplot = ax.pcolormesh(lons, lats, risk, cmap=cmap) #ax.contourf(lons, lats, risk, clevels, cmap=cmap)

cbar = plt.colorbar(cnplot, ticks=cbar_ticks, shrink=0.42, pad = 0.03)

gl = ax.gridlines(crs=projection, linewidth=1, color='gray', alpha=0.5, zorder=200, linestyle='--')

gl.xlabel_style = {'size': 8, 'color': 'gray'}
gl.ylabel_style = {'size': 8, 'color': 'gray'}

gl.bottom_labels = True
gl.left_labels = True

ax.set_extent(zoom_coords)